Importamos las librerías

In [ ]:
import timeit
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json

from numpy import expand_dims
from pytorch_lightning import Trainer
from matplotlib import pyplot
from PIL import Image
from torch.utils.data import DataLoader

from anomalib.models import get_model
from anomalib.models.padim.lightning_model import Padim
from anomalib.config import get_configurable_parameters
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks
from anomalib.data.folder import Folder

En esta función configuramos los parámetros e hiperparámetros de nuestro pryecto: el modelo, los datos, callbacks, la información en el archivo .yaml, etc.

In [ ]:
def anomaset(model):
    CONFIG_PATH = f"./anomalib/models/{model}/custom.yaml"
    # pass the config file to model, callbacks and datamodule
    config = get_configurable_parameters(config_path=CONFIG_PATH)
    
    #config["dataset"]["name"] = "small_18" 
    #config["model"]["backbone"] = "resnet18"  #resnet18, wide_resnet50_2, cait_m48_448, deit_base_distilled_patch16_384
    datamodule = Folder(
        root="./data/naranjas_pynq_2_crop_augmentation",
        image_size=288,
        #task='classification',
        seed=42)

    #datamodule = MVTec(
     #   root="../data/mvtec_anomaly_detection",
     #   category='capsule',
     #   image_size=256,
     #   train_batch_size=32,
     #   test_batch_size=32,
     #   num_workers=8,
     #   task ="segmentation",
     #   seed = 42)

    datamodule.setup()
    #model = Padim(input_size=[288, 288], backbone="wide_resnet50_2", layers=['layer1','layer2','layer3'],pre_trained=True)
    model = get_model(config) # (288, 352)
    callbacks = get_callbacks(config)

    return datamodule, model, callbacks, config

Con la siguientes funciones entrenamos y evaluamos el modelo deseado.

In [ ]:
def anomatrain(datamodule, model, callbacks, config):
    # start training
    trainer = Trainer(**config.trainer, callbacks=callbacks)
    trainer.fit(model=model, datamodule=datamodule)
    # load best model from checkpoint before evaluating
    load_model_callback = LoadModelCallback(weights_path=trainer.checkpoint_callback.best_model_path)
    trainer.callbacks.insert(0, load_model_callback)
    return trainer

In [ ]:
def anomatest(trainer, model, datamodule):
    trainer.test(model=model, datamodule=datamodule)
    df = trainer.logged_metrics
    return df

In [ ]:
def anomastudy(model):
    datamodule, model, callbacks, config = anomaset(model)

    trainer = anomatrain(datamodule, model, callbacks, config)
    
    metrics = anomatest(trainer, model, datamodule)

    return metrics

In [ ]:
def anomaresults(models):
    colection={}
    for model in models:
        metrics = anomastudy(model)
        colection[model] = metrics
    return colection

A pesar de que el código esta preparado para trabajar con una lista de modelos, por limitaciones computacionales solo se ha podido trabajar con un modelo a la vez.

In [ ]:
models = ['padim']#,'draem','dfm', 'cflow', 'stfpm', 'ganomaly', 'dfkde', 'patchcore']

colection = anomaresults(models)
colection['padim']